In [ ]:
!pip install rpy2
from rpy2.robjects.packages import importr
from rpy2.robjects.conversion import localconverter
from rpy2.robjects import pandas2ri
import rpy2.robjects as ro
%load_ext rpy2.ipython

import warnings 
warnings.filterwarnings("ignore")

def dixon_test(X):
    
    with localconverter(ro.default_converter + pandas2ri.converter):
      r_from_pd_df = ro.conversion.py2rpy(X)
   
    result = ro.r.Dixon_MD(r_from_pd_df) #Run Dixon´s test
       
    return result

In [ ]:
"""R code to perform Dixon´s test on data."""

ro.r('''
Dixon_MD<-function(data){
  
  num = dim(data)[2]
  cont0 = 0
  cont1 = 0
  confidence = 0
  i = 1
  RES=rep(NA, dim(data)[2])
  data_temp = matrix(ncol = 1, nrow = dim(data)[1])
  data_temp = data.frame(data_temp)
  result = ''
  while(i <= num) {
    
    data_test = data
    data_temp = data_test[c(i)]
    data_test = data_test[ -c(i) ]

    for (t in 1:dim(data_test)[2]){

      RES[t]=t.test(data_test[which(is.na(data_temp[,1])==T),t],data_test[which(is.na(data_temp[,1])==F),t], paired = FALSE, var.equal = FALSE)$p.value
    } 
    
    RES=RES[1:dim(data_test)[2]]

    if (length(which(RES>0.05))==length(RES)) {
      RES=rep(1,length(RES))
      cont1 = cont1 + 1
    } else {
      RES=rep(0,length(RES))
      cont0 = cont0 + 1
    }
    #RES 1/0 --> 1 if MD are considered to be MCAR and 0 if MD are considered to be MAR
    
    i = i + 1
     
  }

  if (cont1>cont0){
    confidence = cont1/num
    #result  = paste('The Missing Mechanism is MCAR. Confidence: ', confidence)
    result = list('The Missing Mechanism is MCAR.', confidence)
  } else{
    result = list('The Missing Mechanism is not MCAR.')
  }
  
  return (result)
  
}
''')